# Name : Syeda Areeba Nadeem
# Roll No.: 21I-0307
# Section: BS(AI)-K

In [1]:
import pandas as pd
import numpy as np
import sklearn
import nltk
import os
import re
import string
import pickle

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler

from skimage import filters
from skimage import feature
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.io import imread, imshow

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler


from sklearn.pipeline import make_pipeline

import warnings
warnings.filterwarnings('ignore')

# from PIL import Image
# from pytesseract import pytesseract
# !pip install pytesseract
# !pip install pillow

In [2]:
DF = pd.read_csv('labels.csv')
DF.info()
DF

DF.fillna(0,inplace = True)
df = DF[['image_name','text_corrected','overall_sentiment']]
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6992 entries, 0 to 6991
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         6992 non-null   int64 
 1   image_name         6992 non-null   object
 2   text_ocr           6831 non-null   object
 3   text_corrected     6987 non-null   object
 4   overall_sentiment  6992 non-null   object
dtypes: int64(1), object(4)
memory usage: 273.2+ KB


image_name           0
text_corrected       0
overall_sentiment    0
dtype: int64

In [3]:
temp = df[df['text_corrected']== 0]

for i in temp.index:
    print(DF.iloc[i]['text_ocr'])
    if(DF.iloc[i]['text_ocr']!=0):  #check if the ocr column does not has a null value
        df.iloc[i]['text_corrected'] = DF.iloc[i]['text_ocr']

CHALLENGE ACCEPTED! Friend: You can't honestly watch How I Met Your Mother again for like the 4th time..." Me:
0
0
0
0


# PreProcessing

In [4]:
def text_prepocessing(string_text):
  
    lemmatizer = WordNetLemmatizer()
    stop_words=set(stopwords.words('english'))
    
    text = re.sub(r"\S+.com",'', string_text)  # remove instances like imgflip.com
    text = re.sub(r"www.\S+",'', text)  # remove links www.facebook.com -> might not need this, above one can also do that
    text = re.sub(r"http\S+",'', text)  # remove urls 

    text = re.sub("(@[A-Za-z]+)|(#[A-Za-z]+)",'', text)
    text = re.sub("(@)|(#)",'', text)  #to remove hashtags like this => #10 to this => 10
    
    x = [ch for ch in text if ch not in string.punctuation]
    text = ''.join(x)
    clean = [word.lower() for word in text.split() if word.lower() not in stop_words]
    text =  ' '.join(clean)
    
    x = [lemmatizer.lemmatize(word) for word in text]
    text =  ''.join(x)
    
    return text    

In [5]:
df = df[df['text_corrected']!=0]
df_text = df[['text_corrected','image_name']]

for i in range(len(df_text)):
    row_string = df_text.iloc[i]['text_corrected']
#     print(row_string)    
    df_text.iloc[i]['text_corrected'] = text_prepocessing(row_string)

df_text

,text_corrected,image_name
0,look friend lightyear sohalikut trend play 10 ...,image_1.jpg
1,best 10 yearchallenge completed less 4 years k...,image_2.jpeg
2,sam thorne follow follow saw everyone posting ...,image_3.JPG
3,10 year challenge sweet dee edition,image_4.png
4,10 year challenge filter 47 hilarious 10 year ...,image_5.png
...,...,...
6987,tuesday mardi gras wednesday valentines friday...,image_6988.jpg
6988,must watch movies 2017 iti chennai memes maana...,image_6989.jpg
6989,less talking planning soda junk food complaini...,image_6990.png
6990,time fantasy one time unless make time,image_6991.jpg


In [6]:
image_names = list(df_text['image_name'])
df_text.drop('image_name',axis=1,inplace=True)
df_text

,text_corrected
0,look friend lightyear sohalikut trend play 10 ...
1,best 10 yearchallenge completed less 4 years k...
2,sam thorne follow follow saw everyone posting ...
3,10 year challenge sweet dee edition
4,10 year challenge filter 47 hilarious 10 year ...
...,...
6987,tuesday mardi gras wednesday valentines friday...
6988,must watch movies 2017 iti chennai memes maana...
6989,less talking planning soda junk food complaini...
6990,time fantasy one time unless make time


In [7]:
df_text['Sentiment'] = df['overall_sentiment']
order = {'very_positive':1,'positive':1,'neutral':0,'negative':-1,'very_negative':-1}
df_text['Sentiment'] = df_text['Sentiment'].map(order)
df_text

,text_corrected,Sentiment
0,look friend lightyear sohalikut trend play 10 ...,1
1,best 10 yearchallenge completed less 4 years k...,1
2,sam thorne follow follow saw everyone posting ...,1
3,10 year challenge sweet dee edition,1
4,10 year challenge filter 47 hilarious 10 year ...,0
...,...,...
6987,tuesday mardi gras wednesday valentines friday...,0
6988,must watch movies 2017 iti chennai memes maana...,0
6989,less talking planning soda junk food complaini...,1
6990,time fantasy one time unless make time,1


In [36]:
X_train, X_test, y_train, y_test = train_test_split(df_text['text_corrected'],df_text['Sentiment'], test_size=0.24,
                                                    random_state=31)

train = pd.concat([X_train,y_train],axis=1).reset_index()
test = pd.concat([X_test,y_test],axis=1).reset_index()

train.drop(['index'],axis=1,inplace=True)
test.drop(['index'],axis=1,inplace=True)

train.to_csv('Train.csv',index=False)
test.to_csv('Test.csv',index=False)

# Count Vectorizer

In [8]:
#  with open('CV_vectorizer.pkl','wb') as f:
#     pickle.dump(CountVectorizer(),f)

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MaxAbsScaler
from sklearn.decomposition import TruncatedSVD

X = df_text['text_corrected']
y = df_text['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=31)

pipeline = Pipeline(steps=[('count_vec', TfidfVectorizer()), ('scaler', MaxAbsScaler())
                          ,('model', LogisticRegression())])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print("Accuracy",accuracy_score(y_test,y_pred)*100)
print("Recall: ", recall_score(y_test, y_pred, average='macro')*100)
print("Precision: ", precision_score(y_test, y_pred, average='macro')*100)
print('F1 score: ',f1_score(y_test, y_pred, average='macro')*100)
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

Accuracy 54.00572246065808
Recall:  34.08476634592865
Precision:  31.735708698497838
F1 score:  31.93791628822283
Confusion Matrix: 
[[  0  26 102]
 [  3 102 344]
 [ 11 157 653]]


In [ ]:
X = df_text['text_corrected']
y = df_text['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=31)

CV_vectorizer = CountVectorizer()
X_train_CV = CV_vectorizer.fit_transform(X_train)
X_test_CV = CV_vectorizer.transform(X_test)
# print(CV_vectorizer.vocabulary_)
X_train_CV.toarray().shape


In [ ]:
SVM_CV_model = svm.SVC(kernel='linear',C=2)
SVM_CV_model.fit(X_train_CV,y_train)
y_pred = SVM_CV_model.predict(X_test_CV)

print("Accuracy",accuracy_score(y_test,y_pred)*100)
print("Recall: ", recall_score(y_test, y_pred, average='macro')*100)
print("Precision: ", precision_score(y_test, y_pred, average='macro')*100)
print('F1 score: ',f1_score(y_test, y_pred, average='macro')*100)
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

target_names = ['Positive','Neutral', 'Negative']
print(classification_report(y_test,y_pred,target_names=target_names))
SVM_CV_y_pred = y_pred

In [ ]:
LR_CV_model = LogisticRegression(C=8.2,random_state=0).fit(X_train_CV, y_train)
y_pred = LR_CV_model.predict(X_test_CV)

print("Accuracy: ",accuracy_score(y_test,y_pred)*100)
print("Recall: ", recall_score(y_test, y_pred, average='macro')*100)
print("Precision: ", precision_score(y_test, y_pred, average='macro')*100)
print('F1 score: ', f1_score(y_test, y_pred, average='macro')*100)
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

target_names = ['Positive','Neutral', 'Negative']
print(classification_report(y_test,y_pred,target_names=target_names))
LR_CV_y_pred = y_pred

# Extra Trees Classifier

In [11]:
ETC_CV_model = ExtraTreesClassifier(n_estimators=100, random_state=0)
ETC_CV_model.fit(X_train, y_train)

y_pred = ETC_CV_model.predict(X_test)

print("Accuracy: ",accuracy_score(y_test,y_pred)*100)
print("Recall: ", recall_score(y_test, y_pred, average='macro')*100)
print("Precision: ", precision_score(y_test, y_pred, average='macro')*100)
print('F1 score: ',f1_score(y_test, y_pred, average='macro')*100)
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))


target_names = ['Positive','Neutral', 'Negative']
print(classification_report(y_test,y_pred,target_names=target_names))
ETC_CV_y_pred = y_pred

Accuracy:  51.716738197424895
Recall:  34.808498178385314
Precision:  34.61306425104154
F1 score:  34.18165931264592
Confusion Matrix: 
[[  4  35  89]
 [ 16 136 297]
 [ 34 204 583]]
              precision    recall  f1-score   support

    Positive       0.07      0.03      0.04       128
     Neutral       0.36      0.30      0.33       449
    Negative       0.60      0.71      0.65       821

    accuracy                           0.52      1398
   macro avg       0.35      0.35      0.34      1398
weighted avg       0.48      0.52      0.49      1398



# Tf-Idf

In [307]:
X = df_text['text_corrected']
y = df_text['Sentiment']

TFIDF_vectorizer = TfidfVectorizer(max_df=0.75)
X = TFIDF_vectorizer.fit_transform(X)
X.toarray().shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=31)

## LR - TFIDF

In [13]:
LR_TFIDF_model = LogisticRegression(C=131.2,random_state=0).fit(X_train, y_train)
y_pred = LR_TFIDF_model.predict(X_test)

print("Accuracy: ",accuracy_score(y_test,y_pred)*100)
print("Recall: ", recall_score(y_test, y_pred, average='macro')*100)
print("Precision: ", precision_score(y_test, y_pred, average='macro')*100)
print('F1 score: ', f1_score(y_test, y_pred, average='macro')*100)
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

target_names = ['Positive','Neutral', 'Negative']
print(classification_report(y_test,y_pred,target_names=target_names))
LR_TFIDF_y_pred = y_pred

Accuracy:  51.072961373390555
F1 score:  35.033008150222294
Recall:  35.270728242352064
Precision:  35.99649408067987
Confusion Matrix: 
[[  7  28  93]
 [  8 141 300]
 [ 44 211 566]]
              precision    recall  f1-score   support

    Positive       0.12      0.05      0.07       128
     Neutral       0.37      0.31      0.34       449
    Negative       0.59      0.69      0.64       821

    accuracy                           0.51      1398
   macro avg       0.36      0.35      0.35      1398
weighted avg       0.48      0.51      0.49      1398



## SVM - TFIDF

In [14]:
SVM_TFIDF_model = svm.SVC(kernel='linear',C=5)
SVM_TFIDF_model.fit(X_train,y_train)
y_pred = SVM_TFIDF_model.predict(X_test)

print("Accuracy",accuracy_score(y_test,y_pred)*100)
print("Recall: ", recall_score(y_test, y_pred, average='macro')*100)
print("Precision: ", precision_score(y_test, y_pred, average='macro')*100)
print('F1 score: ',f1_score(y_test, y_pred, average='macro')*100)
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

target_names = ['Positive','Neutral', 'Negative']
print(classification_report(y_test,y_pred,target_names=target_names))
SVM_TFIDF_y_pred = y_pred

Accuracy 51.28755364806867
Recall:  35.83449798103242
Precision:  36.92640308246058
Confusion Matrix: 
[[ 11  30  87]
 [ 12 128 309]
 [ 51 192 578]]
F1 score:  35.77741779253909
              precision    recall  f1-score   support

    Positive       0.15      0.09      0.11       128
     Neutral       0.37      0.29      0.32       449
    Negative       0.59      0.70      0.64       821

    accuracy                           0.51      1398
   macro avg       0.37      0.36      0.36      1398
weighted avg       0.48      0.51      0.49      1398



## ETC-TFIDF

In [15]:
ETC_TFIDF_model = ExtraTreesClassifier(n_estimators=100, random_state=0)
ETC_TFIDF_model.fit(X_train, y_train)

y_pred = ETC_TFIDF_model.predict(X_test)

print("Accuracy",accuracy_score(y_test,y_pred)*100)
print("Recall: ", recall_score(y_test, y_pred, average='macro')*100)
print("Precision: ", precision_score(y_test, y_pred, average='macro')*100)
print('F1 score: ',f1_score(y_test, y_pred, average='macro')*100)
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

target_names = ['Positive','Neutral', 'Negative']
print(classification_report(y_test,y_pred,target_names=target_names))
ETC_TFIDF_y_pred = y_pred

Accuracy 53.71959942775394
Recall:  34.71636271947315
Precision:  34.97180840057935
Confusion Matrix: 
[[  3  29  96]
 [  7 106 336]
 [ 26 153 642]]
F1 score:  33.39368570273382
              precision    recall  f1-score   support

    Positive       0.08      0.02      0.04       128
     Neutral       0.37      0.24      0.29       449
    Negative       0.60      0.78      0.68       821

    accuracy                           0.54      1398
   macro avg       0.35      0.35      0.33      1398
weighted avg       0.48      0.54      0.49      1398



# Majority voting for text

In [16]:
model_1 = LR_TFIDF_model
model_2 = SVM_TFIDF_model
model_3 = ETC_TFIDF_model

final_model = VotingClassifier(estimators=[('lr_tfidf', model_1), ('svm_tfidf', model_2), ('etc_tfidf', model_3)], voting='hard')
final_model.fit(X_train, y_train)
pred_final = final_model.predict(X_test)

print("Accuracy: ",accuracy_score(y_test,y_pred)*100)
print("Recall: ", recall_score(y_test, y_pred, average='macro')*100)
print("Precision: ", precision_score(y_test, y_pred, average='macro')*100)
print('F1 score: ',f1_score(y_test, pred_final, average='macro')*100)
f1_text =f1_score(y_test, pred_final, average='macro')*100
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

target_names = ['Positive','Neutral', 'Negative']
print(classification_report(y_test,pred_final,target_names=target_names))

Accuracy:  53.71959942775394
Recall:  34.71636271947315
Precision:  34.97180840057935
Confusion Matrix: 
[[  3  29  96]
 [  7 106 336]
 [ 26 153 642]]
F1 score:  35.255242201339975
              precision    recall  f1-score   support

    Positive       0.14      0.06      0.09       128
     Neutral       0.37      0.28      0.32       449
    Negative       0.59      0.73      0.65       821

    accuracy                           0.52      1398
   macro avg       0.37      0.36      0.35      1398
weighted avg       0.48      0.52      0.49      1398



# Hashing Vectorizer

In [24]:
X = df_text['text_corrected']
y = df_text['Sentiment']

H_vectorizer = HashingVectorizer()
X = H_vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=31)

In [25]:
SVM_HV_model = svm.SVC(kernel='linear',C=5)
SVM_HV_model.fit(X_train,y_train)
y_pred = SVM_HV_model.predict(X_test)

print("Accuracy",accuracy_score(y_test,y_pred)*100)
print("Recall: ", recall_score(y_test, y_pred, average='macro')*100)
print("Precision: ", precision_score(y_test, y_pred, average='macro')*100)
print('F1 score: ',f1_score(y_test, y_pred, average='macro')*100)
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

target_names = ['Positive','Neutral', 'Negative']
print(classification_report(y_test,y_pred,target_names=target_names))
SVM_HV_y_pred = y_pred

Accuracy 51.430615164520745
Recall:  35.460416898381126
Precision:  36.99867161626087
F1 score:  35.31623675391313
Confusion Matrix: 
[[ 10  29  89]
 [ 11 121 317]
 [ 39 194 588]]
              precision    recall  f1-score   support

    Positive       0.17      0.08      0.11       128
     Neutral       0.35      0.27      0.31       449
    Negative       0.59      0.72      0.65       821

    accuracy                           0.51      1398
   macro avg       0.37      0.35      0.35      1398
weighted avg       0.48      0.51      0.49      1398



## LR - Hashing

In [19]:
LR_HV_model = LogisticRegression(C=131.2,random_state=0).fit(X_train, y_train)
y_pred = LR_HV_model.predict(X_test)

print("Accuracy: ",accuracy_score(y_test,y_pred)*100)
print("Recall: ", recall_score(y_test, y_pred, average='macro')*100)
print("Precision: ", precision_score(y_test, y_pred, average='macro')*100)
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

target_names = ['Positive','Neutral', 'Negative']
print('F1 score: ',f1_score(y_test, y_pred, average='macro')*100)
print(classification_report(y_test,y_pred,target_names=target_names))
LR_HV_y_pred = y_pred

Accuracy:  50.35765379113018
Recall:  35.25272586620514
Precision:  35.80417154442039
Confusion Matrix: 
[[  8  26  94]
 [ 14 146 289]
 [ 47 224 550]]
F1 score:  35.130612488088914
              precision    recall  f1-score   support

    Positive       0.12      0.06      0.08       128
     Neutral       0.37      0.33      0.35       449
    Negative       0.59      0.67      0.63       821

    accuracy                           0.50      1398
   macro avg       0.36      0.35      0.35      1398
weighted avg       0.48      0.50      0.49      1398



# IMAGE

The following commented code is for reading images from folder in batches (1000 at a time) extracting thier features and storing them in a dataframe. Later those dataframes are used for ML models

In [20]:
# DF_img = pd.read_csv("labels.csv",index_col=0)
# df_img = DF_img[['image_name','overall_sentiment']]
# df_img

In [21]:
# def canny_feature_extraction(image_path):
#     Image = imread(image_path,as_gray=True)
#     image_resized = resize(Image, (50,50))
#     Canny_img = feature.canny(image_resized)
#     image_features = np.reshape(Canny_img,50*50).astype(int)
#     return image_features

# def sobel_feature_extraction(image_path):
#     Image = imread(image_path,as_gray=True)
#     image_resized = resize(Image, (50,50))
#     Sobel_img = filters.sobel(image_resized)
#     image_features = np.reshape(Sobel_img,50*50)
#     return image_features

# def pixel_feature_extraction(image_path):
#     Image = imread(image_path,as_gray=True)
#     image_resized = resize(Image, (50,50))
#     image_features = np.reshape(image_resized,(50*50))
#     return image_features

In [22]:
# path = './images-6/'

# img_name=[]
# img_sentiments=[]

# df_px = pd.DataFrame(columns=[x for x in range(0,2500)])
# df_sb = pd.DataFrame(columns=[x for x in range(0,2500)])
# df_can = pd.DataFrame(columns=[x for x in range(0,2500)])

# images = os.listdir(path)
# for img in images:
    
#     canny = canny_feature_extraction(path+img)
#     sobel = sobel_feature_extraction(path+img)
#     pixel = pixel_feature_extraction(path+img)
    
#     df_px.loc[len(df_px.index)] = pixel
#     df_sb.loc[len(df_sb.index)] = sobel
#     df_can.loc[len(df_can.index)] = canny
    
#     img_name.append(img) 
    
# #     print(img,"  completed")
#     sentiment = df[df['image_name']==img]['overall_sentiment']
#     img_sentiments.append(sentiment.values[0])
    

In [23]:
# df_can.insert(loc=0, column='Name', value=img_name, allow_duplicates = False)
# df_can.insert(loc=1, column='Sentiment', value=img_sentiments, allow_duplicates = False)

# df_sb.insert(loc=0, column='Name', value=img_name, allow_duplicates = False)
# df_sb.insert(loc=1, column='Sentiment', value=img_sentiments, allow_duplicates = False)

# df_px.insert(loc=0, column='Name', value=img_name, allow_duplicates = False)
# df_px.insert(loc=1, column='Sentiment', value=img_sentiments, allow_duplicates = False)

In [24]:
# df_px.to_csv("pixel-6.csv",index=False)
# df_sb.to_csv("sobel-6.csv",index=False)
# df_can.to_csv("canny-6.csv",index=False)

In [25]:
order = {'very_positive':1,'positive':1,'neutral':0,'negative':-1,'very_negative':-1}

# Pixel

In [237]:
pdf0 = pd.read_csv("pixel-0.csv")
pdf1 = pd.read_csv("pixel-1.csv")
pdf2 = pd.read_csv("pixel-2.csv")
pdf3 = pd.read_csv("pixel-3.csv")
pdf4 = pd.read_csv("pixel-4.csv")
pdf5 = pd.read_csv("pixel-5.csv")
pdf6 = pd.read_csv("pixel-6.csv")

df_pixel = pd.concat([pdf0,pdf1,pdf2,pdf3,pdf4,pdf5,pdf6],axis=0)

img_names = df_pixel['Name']
df_pixel.drop(labels=['Name'],inplace=True,axis=1)

df_pixel['Sentiment'] = df_pixel['Sentiment'].map(order)

# df_pixel

In [238]:
X = df_pixel.drop(labels=['Sentiment'],axis=1)
Y = df_pixel[['Sentiment']]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

## Stochastic Gradient Descent Classifier

In [168]:
SGD_PX_model = make_pipeline(StandardScaler(),SGDClassifier())

SGD_PX_model.fit(X_train, y_train)
y_pred = SGD_PX_model.predict(X_test)

print("Accuracy: ",accuracy_score(y_test,y_pred)*100)
print("Recall: ", recall_score(y_test, y_pred, average='macro')*100)
print("Precision: ", precision_score(y_test, y_pred, average='macro')*100)
print('F1 score: ',f1_score(y_test, y_pred, average='macro')*100)
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

target_names = ['Positive','Neutral', 'Negative']
print(classification_report(y_test,y_pred, target_names=target_names))
SGD_PX_y_pred = y_pred

Accuracy:  48.24874910650465
Recall:  35.110657821816886
Precision:  35.21331863014314
F1 score:  35.0756312332985
Confusion Matrix: 
[[ 13  44  87]
 [ 46 144 254]
 [ 50 243 518]]
              precision    recall  f1-score   support

    Positive       0.12      0.09      0.10       144
     Neutral       0.33      0.32      0.33       444
    Negative       0.60      0.64      0.62       811

    accuracy                           0.48      1399
   macro avg       0.35      0.35      0.35      1399
weighted avg       0.47      0.48      0.47      1399



## MNB - Pixel

In [170]:
MNB_PX_model = MultinomialNB(alpha=16)
MNB_PX_model.fit(X_train, y_train)

y_pred = MNB_PX_model.predict(X_test)

print("Accuracy: ",accuracy_score(y_test,y_pred)*100)
print("Recall: ", recall_score(y_test, y_pred, average='macro')*100)
print("Precision: ", precision_score(y_test, y_pred, average='macro')*100)
print('F1 score: ',f1_score(y_test, y_pred, average='macro')*100)
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

target_names = ['Positive','Neutral', 'Negative']
print(classification_report(y_test,y_pred, target_names=target_names))
MNB_PX_y_pred = y_pred

Accuracy:  44.603288062902074
Recall:  33.979841432060915
Precision:  34.1412936761774
F1 score:  34.0077520183936
Confusion Matrix: 
[[ 17  53  74]
 [ 51 150 243]
 [ 62 292 457]]
              precision    recall  f1-score   support

    Positive       0.13      0.12      0.12       144
     Neutral       0.30      0.34      0.32       444
    Negative       0.59      0.56      0.58       811

    accuracy                           0.45      1399
   macro avg       0.34      0.34      0.34      1399
weighted avg       0.45      0.45      0.45      1399



## KNN - Pixel

In [28]:
KNN_PX_model = KNeighborsClassifier(n_neighbors=3)
KNN_PX_model.fit(X_train, y_train)
y_pred = KNN_PX_model.predict(X_test)

print("Accuracy: ",accuracy_score(y_test,y_pred)*100)
print("Recall: ", recall_score(y_test, y_pred, average='macro')*100)
print("Precision: ", precision_score(y_test, y_pred, average='macro')*100)
print('F1 score: ',f1_score(y_test, y_pred, average='macro')*100)
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

target_names = ['Positive','Neutral', 'Negative']
print(classification_report(y_test,y_pred,target_names=target_names))

KNN_PX_y_pred = y_pred

Accuracy:  47.46247319513939
Recall:  33.85086689648958
Precision:  33.442977355237936
F1 score:  33.04597995010769
Confusion Matrix: 
[[ 20  32  92]
 [ 54  81 309]
 [ 85 163 563]]
              precision    recall  f1-score   support

    Positive       0.13      0.14      0.13       144
     Neutral       0.29      0.18      0.23       444
    Negative       0.58      0.69      0.63       811

    accuracy                           0.47      1399
   macro avg       0.33      0.34      0.33      1399
weighted avg       0.44      0.47      0.45      1399



# SOBEL

In [263]:
s0 = pd.read_csv("sobel-0.csv")
s1 = pd.read_csv("sobel-1.csv")
s2 = pd.read_csv("sobel-2.csv")
s3 = pd.read_csv("sobel-3.csv")
s4 = pd.read_csv("sobel-4.csv")
s5 = pd.read_csv("sobel-5.csv")
s6 = pd.read_csv("sobel-6.csv")

df_sobel = pd.concat([s0,s1,s2,s3,s4,s5,s6],axis=0)

df_sobel.drop(labels=['Name'],inplace=True,axis=1)
df_sobel['Sentiment'] = df_sobel['Sentiment'].map(order)

In [264]:
X = df_sobel.drop(labels=['Sentiment'],axis=1)
Y = df_sobel[['Sentiment']]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

## KNN - Sobel

In [245]:
KNN_SB_model = KNeighborsClassifier(n_neighbors=3)
KNN_SB_model.fit(X_train, y_train)
y_pred = KNN_SB_model.predict(X_test)

print("Accuracy: ",accuracy_score(y_test,y_pred)*100)
print("Recall: ", recall_score(y_test, y_pred, average='macro')*100)
print("Precision: ", precision_score(y_test, y_pred, average='macro')*100)
print('F1 score: ',f1_score(y_test, y_pred, average='macro')*100)
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

target_names = ['Positive','Neutral', 'Negative']
print(classification_report(y_test,y_pred,target_names=target_names))
KNN_SB_y_pred = y_pred

Accuracy:  48.67762687634024
Recall:  34.50856837108378
Precision:  34.48719782294646
F1 score:  33.88910018935066
Confusion Matrix: 
[[ 18  29  97]
 [ 53  91 300]
 [ 77 162 572]]
              precision    recall  f1-score   support

    Positive       0.12      0.12      0.12       144
     Neutral       0.32      0.20      0.25       444
    Negative       0.59      0.71      0.64       811

    accuracy                           0.49      1399
   macro avg       0.34      0.35      0.34      1399
weighted avg       0.46      0.49      0.46      1399



## MNB - sobel

In [258]:
MNB_SB_model = MultinomialNB()
MNB_SB_model.fit(X_train, y_train)

y_pred = MNB_SB_model.predict(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred)*100)
print("Recall: ", recall_score(y_test, y_pred, average='macro')*100)
print("Precision: ", precision_score(y_test, y_pred, average='macro')*100)
print('F1 score: ',f1_score(y_test, y_pred, average='macro')*100)
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

target_names = ['Positive','Neutral', 'Negative']
print(classification_report(y_test,y_pred,target_names=target_names))

MNB_CN_y_pred = y_pred

Accuracy:  49.2494639027877
Recall:  32.24000819561855
Precision:  32.06874540111927
F1 score:  30.895103396549878
Confusion Matrix: 
[[  4  33 107]
 [ 11  93 340]
 [ 24 195 592]]
              precision    recall  f1-score   support

    Positive       0.10      0.03      0.04       144
     Neutral       0.29      0.21      0.24       444
    Negative       0.57      0.73      0.64       811

    accuracy                           0.49      1399
   macro avg       0.32      0.32      0.31      1399
weighted avg       0.43      0.49      0.45      1399



## LDA - Sobel

In [268]:
LDA_SB_model = LinearDiscriminantAnalysis()
LDA_SB_model.fit(X_train, y_train)

y_pred = LDA_SB_model.predict(X_test)

print("Accuracy: ",accuracy_score(y_test,y_pred)*100)
print("Recall: ", recall_score(y_test, y_pred, average='macro')*100)
print("Precision: ", precision_score(y_test, y_pred, average='macro')*100)
print('F1 score: ',f1_score(y_test, y_pred, average='macro')*100)
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

target_names = ['Positive','Neutral', 'Negative']
print(classification_report(y_test,y_pred,target_names=target_names))
LDA_SB_y_pred = y_pred

Accuracy:  44.38884917798428
Recall:  34.59113398231771
Precision:  34.41022543647757
F1 score:  34.408131459138694
Confusion Matrix: 
[[ 23  42  79]
 [ 67 138 239]
 [ 98 253 460]]
              precision    recall  f1-score   support

    Positive       0.12      0.16      0.14       144
     Neutral       0.32      0.31      0.31       444
    Negative       0.59      0.57      0.58       811

    accuracy                           0.44      1399
   macro avg       0.34      0.35      0.34      1399
weighted avg       0.46      0.44      0.45      1399



# CANNY

In [269]:
c0 = pd.read_csv("canny-0.csv")
c1 = pd.read_csv("canny-1.csv")
c2 = pd.read_csv("canny-2.csv")
c3 = pd.read_csv("canny-3.csv")
c4 = pd.read_csv("canny-4.csv")
c5 = pd.read_csv("canny-5.csv")
c6 = pd.read_csv("canny-6.csv")

df_canny = pd.concat([c0,c1,c2,c3,c4,c5,c6],axis=0)

df_canny.drop(labels=['Name'],inplace=True,axis=1)
df_canny['Sentiment'] = df_canny['Sentiment'].map(order)

In [270]:
X = df_canny.drop(labels=['Sentiment'],axis=1)
Y = df_canny[['Sentiment']]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=31)

## LDA - Canny

In [271]:
LDA_CN_model = LinearDiscriminantAnalysis()
LDA_CN_model.fit(X_train, y_train)

y_pred = LDA_CN_model.predict(X_test)

print("Accuracy: ",accuracy_score(y_test,y_pred)*100)
print("Recall: ", recall_score(y_test, y_pred, average='macro')*100)
print("Precision: ", precision_score(y_test, y_pred, average='macro')*100)
print('F1 score: ',f1_score(y_test, y_pred, average='macro')*100)
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

target_names = ['Positive','Neutral', 'Negative']
print(classification_report(y_test,y_pred,target_names=target_names))
LDA_CN_y_pred = y_pred

Accuracy:  45.5325232308792
Recall:  34.663421610287074
Precision:  34.690638534553905
F1 score:  34.512107502555004
Confusion Matrix: 
[[ 18  38  73]
 [ 58 145 256]
 [110 227 474]]
              precision    recall  f1-score   support

    Positive       0.10      0.14      0.11       129
     Neutral       0.35      0.32      0.33       459
    Negative       0.59      0.58      0.59       811

    accuracy                           0.46      1399
   macro avg       0.35      0.35      0.35      1399
weighted avg       0.47      0.46      0.46      1399



## Decision Tree - Canny

In [272]:
DT_CN_model = DecisionTreeClassifier(random_state=0)
DT_CN_model.fit(X_train,y_train)
y_pred = DT_CN_model.predict(X_test)

print("Accuracy: ",accuracy_score(y_test,y_pred)*100)
print("Recall: ", recall_score(y_test, y_pred, average='macro')*100)
print("Precision: ", precision_score(y_test, y_pred, average='macro')*100)
print('F1 score: ',f1_score(y_test, y_pred, average='macro')*100)
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

target_names = ['Positive','Neutral', 'Negative']
print(classification_report(y_test,y_pred,target_names=target_names))

DT_CN_y_pred = y_pred

Accuracy:  45.1036454610436
Recall:  33.64218719898685
Precision:  33.77424836450078
F1 score:  33.63865906194199
Confusion Matrix: 
[[ 14  38  77]
 [ 64 148 247]
 [ 92 250 469]]
              precision    recall  f1-score   support

    Positive       0.08      0.11      0.09       129
     Neutral       0.34      0.32      0.33       459
    Negative       0.59      0.58      0.58       811

    accuracy                           0.45      1399
   macro avg       0.34      0.34      0.34      1399
weighted avg       0.46      0.45      0.46      1399



# Random Forest

In [273]:
RFC_CN_model = RandomForestClassifier(n_estimators=8,random_state=0)
RFC_CN_model.fit(X_train, y_train)
y_pred = RFC_CN_model.predict(X_test)

print("Accuracy: ",accuracy_score(y_test,y_pred)*100)
print("Recall: ", recall_score(y_test, y_pred, average='macro')*100)
print("Precision: ", precision_score(y_test, y_pred, average='macro')*100)
print('F1 score: ',f1_score(y_test, y_pred, average='macro')*100)
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

target_names = ['Positive','Neutral', 'Negative']
print(classification_report(y_test,y_pred,target_names=target_names))

RFC_CN_y_pred = y_pred

Accuracy:  50.964974982130094
Recall:  34.6470826260517
Precision:  35.16679821693015
F1 score:  34.04414344171744
Confusion Matrix: 
[[  5  34  90]
 [ 12 135 312]
 [ 28 210 573]]
              precision    recall  f1-score   support

    Positive       0.11      0.04      0.06       129
     Neutral       0.36      0.29      0.32       459
    Negative       0.59      0.71      0.64       811

    accuracy                           0.51      1399
   macro avg       0.35      0.35      0.34      1399
weighted avg       0.47      0.51      0.48      1399



## Multinomial NB - Canny

In [230]:
MNB_CN_model = MultinomialNB()
MNB_CN_model.fit(X_train, y_train)

y_pred = MNB_CN_model.predict(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred)*100)
print("Recall: ", recall_score(y_test, y_pred, average='macro')*100)
print("Precision: ", precision_score(y_test, y_pred, average='macro')*100)
print('F1 score: ',f1_score(y_test, y_pred, average='macro')*100)
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

target_names = ['Positive','Neutral', 'Negative']
print(classification_report(y_test,y_pred,target_names=target_names))
MNB_CN_y_pred = y_pred

Accuracy:  46.89063616869193
Recall:  33.541689742930885
Precision:  33.598270226269946
F1 score:  33.427107009591325
Confusion Matrix: 
[[ 12  35  82]
 [ 56 126 277]
 [ 78 215 518]]
              precision    recall  f1-score   support

    Positive       0.08      0.09      0.09       129
     Neutral       0.34      0.27      0.30       459
    Negative       0.59      0.64      0.61       811

    accuracy                           0.47      1399
   macro avg       0.34      0.34      0.33      1399
weighted avg       0.46      0.47      0.46      1399



# Majority voting for image

In [275]:
model_1 = LDA_CN_model
model_2 = RFC_CN_model
model_3 = MNB_CN_model

final_model = VotingClassifier(estimators=[('lda_cn', model_1), ('dt_cn', model_2), ('mnb_cn', model_3)], voting='hard')
final_model.fit(X_train, y_train)
pred_final = final_model.predict(X_test)

print("Accuracy: ",accuracy_score(y_test,y_pred)*100)
print("Recall: ", recall_score(y_test, y_pred, average='macro')*100)
print("Precision: ", precision_score(y_test, y_pred, average='macro')*100)
print('F1 score: ',f1_score(y_test, y_pred, average='macro')*100)
f1_img=f1_score(y_test, y_pred, average='macro')*100
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

target_names = ['Positive','Neutral', 'Negative']
print(classification_report(y_test,pred_final,target_names=target_names))

Accuracy:  50.964974982130094
Recall:  34.6470826260517
Precision:  35.16679821693015
F1 score:  34.04414344171744
Confusion Matrix: 
[[  5  34  90]
 [ 12 135 312]
 [ 28 210 573]]
              precision    recall  f1-score   support

    Positive       0.09      0.10      0.09       129
     Neutral       0.37      0.24      0.29       459
    Negative       0.59      0.69      0.64       811

    accuracy                           0.49      1399
   macro avg       0.35      0.35      0.34      1399
weighted avg       0.47      0.49      0.47      1399



In [47]:
# print(LDA_CN_y_pred)
# print(DT_CN_y_pred)
# print(MNB_CN_y_pred)

# print(LR_TFIDF_y_pred)
# print(SVM_TFIDF_y_pred)
# print(ETC_TFIDF_y_pred)

# print(len(LDA_CN_y_pred))
# print(len(DT_CN_y_pred))
# print(len(MNB_CN_y_pred))

# print(len(LR_TFIDF_y_pred))
# print(len(SVM_TFIDF_y_pred))
# print(len(ETC_TFIDF_y_pred))

In [2]:
final = (f1_img+f1_text)/2
final 

34.6496928215287